# Goal - 4) Error Detection & Data Validation 
- Detecting & marking missing values
- Removing duplicate records
- Validating domain specific rules
- Removing broken or corrupted rows n or corrupted rows 

In [1]:
# 4) Error Detection & Data Validation - Dataset

import pandas as pd
import numpy as np

# Create the dataset
data = {
    'customer_id': [2001, 2002, 2002, 2004, 2005, 2006, 2007],
    'age': [25, 32, 32, -5, 40, None, 29],
    'email': ['alice@example.com', 'bob@example.com', 'bob@example.com', None, 'eve@@example.com', 'frank@example.com', 'grace@example'],
    'signup_date': ['2021-01-10', 'not_a_date', 'not_a_date', '2021-03-15', '2021-04-01', '2021-04-22', None],
    'monthly_spend': [100.0, 200.0, 200.0, 'N/A', 500.0, 'five hundred', 300.0],
    'status': ['active', 'inactive', 'inactive', 'unknown', 'active', np.nan, 'active']
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df

,customer_id,age,email,signup_date,monthly_spend,status
0,2001,25.0,alice@example.com,2021-01-10,100.0,active
1,2002,32.0,bob@example.com,not_a_date,200.0,inactive
2,2002,32.0,bob@example.com,not_a_date,200.0,inactive
3,2004,-5.0,None,2021-03-15,N/A,unknown
4,2005,40.0,eve@@example.com,2021-04-01,500.0,active
5,2006,NaN,frank@example.com,2021-04-22,five hundred,NaN
6,2007,29.0,grace@example,None,300.0,active


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   customer_id    7 non-null      int64  
 1   age            6 non-null      float64
 2   email          6 non-null      object 
 3   signup_date    6 non-null      object 
 4   monthly_spend  7 non-null      object 
 5   status         6 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 468.0+ bytes


#### 4-1) Detecting & Marking Missing Values = 
isnull(), isna(), notnull(), info(), df.describe(),
df.isnull().sum()

In [3]:
# isnull() & isna() both are same
# notnull() & notna both are same

df.isnull() # Detects missing (NaN) values & gives True for null/missing values

,customer_id,age,email,signup_date,monthly_spend,status
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,True,False,False,False
4,False,False,False,False,False,False
5,False,True,False,False,False,True
6,False,False,False,True,False,False


In [4]:
df.notnull() # Detects non-missing values & gives False for missing values, opposite to isnull

,customer_id,age,email,signup_date,monthly_spend,status
0,True,True,True,True,True,True
1,True,True,True,True,True,True
2,True,True,True,True,True,True
3,True,True,False,True,True,True
4,True,True,True,True,True,True
5,True,False,True,True,True,False
6,True,True,True,False,True,True


In [5]:
# df.info()	= Shows non-null counts and column types	
df.info()

# df.describe() = Skips nulls in summary — helps spot what's missing
# Count shows how many non-null values exist in each column — a good way to spot missing data.
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   customer_id    7 non-null      int64  
 1   age            6 non-null      float64
 2   email          6 non-null      object 
 3   signup_date    6 non-null      object 
 4   monthly_spend  7 non-null      object 
 5   status         6 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 468.0+ bytes


,customer_id,age
count,7.000000,6.000000
mean,2003.857143,25.500000
std,2.267787,15.732133
min,2001.000000,-5.000000
25%,2002.000000,26.000000
50%,2004.000000,30.500000
75%,2005.500000,32.000000
max,2007.000000,40.000000


In [6]:
# df.isnull().sum()	= Shows count of missing values per column	
df.isnull().sum()  # mean, value_counts etc.

customer_id      0
age              1
email            1
signup_date      1
monthly_spend    0
status           1
dtype: int64

In [7]:
df.isnull().value_counts()

customer_id  age    email  signup_date  monthly_spend  status
False        False  False  False        False          False     4
                           True         False          False     1
                    True   False        False          False     1
             True   False  False        False          True      1
Name: count, dtype: int64

##### df.isnull().value_counts() gives dataframe of row wise pattern of missing values

- it first(in 1st row) gives count of those rows which dont have any missing values, All False.
- Then from 2nd row it starts counting each unique pattern of missing values row wise.
- In short -
     - df.isnull() gives a True/False mask of missing values.
     - .value_counts() then counts how often each pattern of nulls appears across rows.
- It helps in quickly see null patterns across the dataset — especially useful for:
     - Spotting unusual rows like How many rows have missing data.
     - Deciding how to impute or drop
     - Creating custom data cleaning rules like Where to apply cleaning rules (e.g., drop rows where age and status are both missing)
     - Shows which columns are jointly missing
- .value_counts() actually counts how many rows share the same combination of missing/non-missing values across all columns.

##### Once we get the null patterns from df.isnull().value_counts(), we can make smart, targeted decisions about how to clean or handle missing data

- 1. Drop fully empty rows = If any pattern shows all columns are True (null) → those rows contain zero useful data.df.dropna(how='all')
  2. Drop specific columns if they’re missing too often.
  3. Impute values for key columns = If only a few values are missing in important columns.
  4. Investigate suspicious null combinations
  5. Custom cleanup by null pattern
- All these things can be traced using different codes but this way we can have a big picture of missing values. 

In [8]:
df.isnull().mean()

# This is a quick and powerful command that tells you the percentage of missing values in each column.
# Helps to quickly identify which columns have too many nulls
# LHelps to set thresholds for dropping or imputing

customer_id      0.000000
age              0.142857
email            0.142857
signup_date      0.142857
monthly_spend    0.000000
status           0.142857
dtype: float64

In [9]:
# To get % instead of fraction
df.isnull().mean() * 100

customer_id       0.000000
age              14.285714
email            14.285714
signup_date      14.285714
monthly_spend     0.000000
status           14.285714
dtype: float64

In [10]:
df.isnull().any(axis=1) # this gives series of True/False which tells presence of missing values row wise.

0    False
1    False
2    False
3     True
4    False
5     True
6     True
dtype: bool

In [11]:
df[df.isnull().any(axis=1)]  # axis=1 for rows

# It is used to filter and display only the rows that contain at least one missing value (NaN) in any column.
# Breakdown:
      # df.isnull() -> returns a DataFrame of True/False values for each cell (True if the value is missing).
      # .any(axis=1) -> checks row-wise: returns True if any column in that row has a null.
      # df[...] -> selects only those rows where the condition is True.
# Use case: Quickly check which rows are incomplete and may need: 1) Imputation 2) Removal 3) Manual inspection

,customer_id,age,email,signup_date,monthly_spend,status
3,2004,-5.0,None,2021-03-15,N/A,unknown
5,2006,NaN,frank@example.com,2021-04-22,five hundred,NaN
6,2007,29.0,grace@example,None,300.0,active


In [12]:
# df[df.isnull().any(axis=0)] -> Using axis=0 is not not valid for filtering rows and will raise an error or give unexpected results.

# although we can use below : # axis=0 for columns
df.isnull().any(axis=0) # it tells whether a particular column contains any missing value or not

customer_id      False
age               True
email             True
signup_date       True
monthly_spend    False
status            True
dtype: bool

In [13]:
# all rules applied to 'any' are applicable to 'all'
# all gives True if all values are missing/NaN
df.isnull().all(axis=1) # for rows 
df.isnull().all(axis=0) # for columns

customer_id      False
age              False
email            False
signup_date      False
monthly_spend    False
status           False
dtype: bool

#### 4-2) Removing Duplicate Records =
- df.drop_duplicates(), df.loc[~df.duplicated()], numpy.unique() or set()
- groupby().agg() as deduping logic
- drop_duplicates() in Dask

In [14]:
# df.drop_duplicates() = Standard method in pandas to drop duplicate rows.
df.drop_duplicates() # Removes duplicate rows across all columns, keeping the first occurrence by default.

# df.drop_duplicates(subset=['col1', 'col2'], keep='first')
#    --> subset = Check duplicates only on selected columns
#    --> keep='first' = Keep first occurrence (default)
#    --> keep='last' = Keep last occurrence
#    --> keep=False	= Drop all duplicates (no row retained)

# To drop duplicates case-insensitively (e.g., treat 'Alice', 'alice', 'ALICE' as the same), 
# You need to normalize the text before applying drop_duplicates().

,customer_id,age,email,signup_date,monthly_spend,status
0,2001,25.0,alice@example.com,2021-01-10,100.0,active
1,2002,32.0,bob@example.com,not_a_date,200.0,inactive
3,2004,-5.0,None,2021-03-15,N/A,unknown
4,2005,40.0,eve@@example.com,2021-04-01,500.0,active
5,2006,NaN,frank@example.com,2021-04-22,five hundred,NaN
6,2007,29.0,grace@example,None,300.0,active


In [15]:
df.duplicated() # gives True for first occurence of duplicate rows

0    False
1    False
2     True
3    False
4    False
5    False
6    False
dtype: bool

In [16]:
# filter and give back only non duplicated/unique rows
df.loc[~df.duplicated()]

# df.loc[~df.duplicated()] = Returns True for duplicate rows (after the first occurrence).
# ~ → Negates the result (so True becomes False and vice versa).

,customer_id,age,email,signup_date,monthly_spend,status
0,2001,25.0,alice@example.com,2021-01-10,100.0,active
1,2002,32.0,bob@example.com,not_a_date,200.0,inactive
3,2004,-5.0,None,2021-03-15,N/A,unknown
4,2005,40.0,eve@@example.com,2021-04-01,500.0,active
5,2006,NaN,frank@example.com,2021-04-22,five hundred,NaN
6,2007,29.0,grace@example,None,300.0,active


In [17]:
# it gives only duplicate rows
df[df.duplicated()]

,customer_id,age,email,signup_date,monthly_spend,status
2,2002,32.0,bob@example.com,not_a_date,200.0,inactive


In [18]:
# numpy.unique() or set() = Use for 1D arrays or single columns to get unique values.
np.unique(df['customer_id'])      # Returns sorted unique values.

array([2001, 2002, 2004, 2005, 2006, 2007], dtype=int64)

In [19]:
# set(df['col'])  # Faster, unsorted unique values. gives set of unique values. order of values changes everytime.
set(df['age'])

{32.0, 40.0, nan, -5.0, 25.0, 29.0}

In [20]:
# Using groupby : deduplicate dataframe by grouping and then aggregate data meaningfully.

df.groupby('customer_id').agg('first').reset_index()

# What it does:
    # groupby('customer_id'): Groups the DataFrame by the 'customer_id' column.
    # .agg('first'): From each group, picks the first non-null value in every column.
    # .reset_index(): Converts the 'customer_id' (which becomes index during grouping) back into a regular column.

# Why use this?
# We don’t just drop duplicate rows — we are saying: Give me one row per customer_id, and for other columns, just take the first valid value.
# Usually it is preferred when .drop_duplicates() is not enough and we want more control over how each column is handled during deduplication.

# This is helpful when:
    # We have duplicate ids.
    # We want to keep one row per id (but not blindly drop rows).
    # We want to aggregate other columns intelligently — like picking the first, last, max, min, or average.

,customer_id,age,email,signup_date,monthly_spend,status
0,2001,25.0,alice@example.com,2021-01-10,100.0,active
1,2002,32.0,bob@example.com,not_a_date,200.0,inactive
2,2004,-5.0,None,2021-03-15,N/A,unknown
3,2005,40.0,eve@@example.com,2021-04-01,500.0,active
4,2006,NaN,frank@example.com,2021-04-22,five hundred,None
5,2007,29.0,grace@example,None,300.0,active


In [21]:
# we can also use custom logic in groupby instead of 'first', different logic for different columns.

df.groupby('customer_id').agg({
    'status': 'last',
    'age': 'mean'
}).reset_index()

,customer_id,status,age
0,2001,active,25.0
1,2002,inactive,32.0
2,2004,unknown,-5.0
3,2005,active,40.0
4,2006,None,NaN
5,2007,active,29.0


##### drop_duplicates() in Dask = Used for large datasets that don’t fit in memory.
- import dask.dataframe as dd
- ddf = dd.from_pandas(df, npartitions=2) # npartitions	Number of chunks Dask divides data into
- ddf = ddf.drop_duplicates().compute() # .compute()	executes the entire task and gives a real pandas DataFrame result.

#### 4-3) Validating Domain-Specific Rules = 
- apply() (with custom logic)
- query()
- between()
- logical conditions (df[col] < 0)
- assert statements
- Boolean masking 

In [22]:
# apply() with custom logic	= Apply row-wise or column-wise validation
df['age'].apply(lambda x: x < 0)


0    False
1    False
2    False
3     True
4    False
5    False
6    False
Name: age, dtype: bool

In [23]:
# query() = Filter rows based on a rule (SQL-like syntax)	
df.query("age >= 18 and age <= 60")

,customer_id,age,email,signup_date,monthly_spend,status
0,2001,25.0,alice@example.com,2021-01-10,100.0,active
1,2002,32.0,bob@example.com,not_a_date,200.0,inactive
2,2002,32.0,bob@example.com,not_a_date,200.0,inactive
4,2005,40.0,eve@@example.com,2021-04-01,500.0,active
6,2007,29.0,grace@example,None,300.0,active


In [24]:
# between()	= Check if values fall within a range	
# all values should be numerical with data type
df['age'].between(25, 32)  # 25 & 32 both are inclusive

# with masking
df[df['age'].between(25,32)]

,customer_id,age,email,signup_date,monthly_spend,status
0,2001,25.0,alice@example.com,2021-01-10,100.0,active
1,2002,32.0,bob@example.com,not_a_date,200.0,inactive
2,2002,32.0,bob@example.com,not_a_date,200.0,inactive
6,2007,29.0,grace@example,None,300.0,active


In [25]:
# Logical conditions = Use boolean expressions for rules
df[df['age'] < 0]

,customer_id,age,email,signup_date,monthly_spend,status
3,2004,-5.0,None,2021-03-15,N/A,unknown


##### assert statements	= Raise error if rule is violated	
assert (df['age'] < 40).all(), "Invalid score!"

##### This checks: "Are all values in age column < 40?"
    # If yes -> Nothing happens (silent pass)
    # If no -> Raises an AssertionError with the message: "Invalid score!"

##### Why use it? -> To validate assumptions in your data before processing. Example use cases:
    # Age should not exceed 40.
    # Spend amount should not be negative.
    # Dates should not be in the future.

##### It raises error like below :
---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
Cell In[27], line 1
----> 1 assert (df['age'] < 40).all(), "Invalid score!"

AssertionError: Invalid score!

In [26]:
# Boolean masking = Identify or select rows that violate the rule	
mask = df['age'] < 0; df[mask]

,customer_id,age,email,signup_date,monthly_spend,status
3,2004,-5.0,None,2021-03-15,N/A,unknown


In [27]:
# Example Use Case: Validate that customer_id is positive and below 2005
# Boolean mask
df[~df['customer_id'].between(0, 2005)]

,customer_id,age,email,signup_date,monthly_spend,status
5,2006,NaN,frank@example.com,2021-04-22,five hundred,NaN
6,2007,29.0,grace@example,None,300.0,active


In [28]:
# assert
assert df['customer_id'].between(0, 2005).any(), "customer_id out of valid range!"

# if it satisfies & validate the condition, nothing gives in output
# if it does not then it gives error with above mentioned error message.

In [29]:
# apply
df['age_flag'] = df['age'].apply(lambda x: 'Invalid' if x < 0 else 'Valid'); df['age_flag']

0      Valid
1      Valid
2      Valid
3    Invalid
4      Valid
5      Valid
6      Valid
Name: age_flag, dtype: object

#### 4-4) Removing Broken or Corrupted Rows =
- pd.to_numeric(..., errors='coerce')
- dropna()
- try-except inside apply()
- Row-wise validation functions

In [30]:
# pd.to_numeric(..., errors='coerce') = Converts bad strings to NaN	
df['monthly_spend'] = pd.to_numeric(df['monthly_spend'], errors = 'coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   customer_id    7 non-null      int64  
 1   age            6 non-null      float64
 2   email          6 non-null      object 
 3   signup_date    6 non-null      object 
 4   monthly_spend  5 non-null      float64
 5   status         6 non-null      object 
 6   age_flag       7 non-null      object 
dtypes: float64(2), int64(1), object(4)
memory usage: 524.0+ bytes


In [31]:
df.dropna(subset=['age']) # it drops rows with NaN after coercion

# It only look at given column to decide which rows to drop.
# Other columns may still have NaNs — they won’t affect this filtering.
# it drop that entire row, helps when we dont want a single NaN value in that column.
# for all NaN use = df.dropna(subset=['age'],how='all')

,customer_id,age,email,signup_date,monthly_spend,status,age_flag
0,2001,25.0,alice@example.com,2021-01-10,100.0,active,Valid
1,2002,32.0,bob@example.com,not_a_date,200.0,inactive,Valid
2,2002,32.0,bob@example.com,not_a_date,200.0,inactive,Valid
3,2004,-5.0,None,2021-03-15,NaN,unknown,Invalid
4,2005,40.0,eve@@example.com,2021-04-01,500.0,active,Valid
6,2007,29.0,grace@example,None,300.0,active,Valid


In [32]:
# try-except inside apply() = Catch and handle conversion or logic errors per row	
# Apply custom logic safely to avoid crash

def safe_convert(x):
    try:
        return float(x)
    except:
        return np.nan

df['monthly_spend'] = df['monthly_spend'].apply(safe_convert).dropna()
df

,customer_id,age,email,signup_date,monthly_spend,status,age_flag
0,2001,25.0,alice@example.com,2021-01-10,100.0,active,Valid
1,2002,32.0,bob@example.com,not_a_date,200.0,inactive,Valid
2,2002,32.0,bob@example.com,not_a_date,200.0,inactive,Valid
3,2004,-5.0,None,2021-03-15,NaN,unknown,Invalid
4,2005,40.0,eve@@example.com,2021-04-01,500.0,active,Valid
5,2006,NaN,frank@example.com,2021-04-22,NaN,NaN,Valid
6,2007,29.0,grace@example,None,300.0,active,Valid


In [33]:
# its not showing difference because Non numeric values in monthly_spend column were already manages earlier
# lets create new dataset with different name for this

# Create the dataset
data = {
    'customer_id': [2001, 2002, 2002, 2004, 2005, 2006, 2007],
    'age': [25, 32, 32, -5, 40, None, 29],
    'email': ['alice@example.com', 'bob@example.com', 'bob@example.com', None, 'eve@@example.com', 'frank@example.com', 'grace@example'],
    'signup_date': ['2021-01-10', 'not_a_date', 'not_a_date', '2021-03-15', '2021-04-01', '2021-04-22', None],
    'monthly_spend': [100.0, 200.0, 200.0, 'N/A', 500.0, 'five hundred', 300.0],
    'status': ['active', 'inactive', 'inactive', 'unknown', 'active', np.nan, 'active']
}

# Convert to DataFrame
df1 = pd.DataFrame(data)

# now apply

def safe_convert1(y):
    try:
        return float(y)
    except:
        return np.nan

df1['monthly_spend'] = df1['monthly_spend'].apply(safe_convert1)
df1

# it has converted no numeric values of monthly_spend column into NaN & keeps numeric values as it is
# it also changes the data type of that column to float64

,customer_id,age,email,signup_date,monthly_spend,status
0,2001,25.0,alice@example.com,2021-01-10,100.0,active
1,2002,32.0,bob@example.com,not_a_date,200.0,inactive
2,2002,32.0,bob@example.com,not_a_date,200.0,inactive
3,2004,-5.0,None,2021-03-15,NaN,unknown
4,2005,40.0,eve@@example.com,2021-04-01,500.0,active
5,2006,NaN,frank@example.com,2021-04-22,NaN,NaN
6,2007,29.0,grace@example,None,300.0,active


In [35]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   customer_id    7 non-null      int64  
 1   age            6 non-null      float64
 2   email          6 non-null      object 
 3   signup_date    6 non-null      object 
 4   monthly_spend  5 non-null      float64
 5   status         6 non-null      object 
dtypes: float64(2), int64(1), object(3)
memory usage: 468.0+ bytes


In [39]:
# Row-wise validation functions = Flag/remove rows that violate custom rules	
# Custom row validation function
def is_valid_row(row):
    return (
        30 <= row['age'] <= 40 and
        row['monthly_spend'] >= 200 and
        '@' in row['email']
    )

# Apply validation function row-wise and keep only valid rows
df_cleaned = df1[df1.apply(is_valid_row, axis=1)]; df_cleaned

,customer_id,age,email,signup_date,monthly_spend,status
1,2002,32.0,bob@example.com,not_a_date,200.0,inactive
2,2002,32.0,bob@example.com,not_a_date,200.0,inactive
4,2005,40.0,eve@@example.com,2021-04-01,500.0,active
